In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from numpy import zeros, ones, eye
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
INPUTS = tf.constant([[0,0],[0,1],[1,0],[1,1]], dtype=tf.float32)
AND = tf.constant([[0],[0],[0],[1]], dtype=tf.float32) # also tf.float32 implicitly
OR = tf.constant([[0],[1],[1],[1]], dtype=tf.float32) # also tf.float32 implicitly
NOR = tf.constant([[1],[0],[0],[0]], dtype=tf.float32) # also tf.float32 implicitly
NAND = tf.constant([[1],[1],[1],[0]], dtype=tf.float32) # also tf.float32 implicitly
XOR = tf.constant([[0],[1],[1],[0]], dtype=tf.float32) # also tf.float32 implicitly
total = INPUTS + AND
print(INPUTS)
print(AND)
print(total)

Tensor("Const:0", shape=(4, 2), dtype=float32)
Tensor("Const_1:0", shape=(4, 1), dtype=float32)
Tensor("add:0", shape=(4, 2), dtype=float32)


In [3]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())
writer.flush()
# run on the terminal: 
# tensorboard --logdir .

In [4]:
sess = tf.Session()
print(sess.run(total))
print(sess.run({'AND OR':(AND, OR), 'total':total}))

[[0. 0.]
 [0. 1.]
 [1. 0.]
 [2. 2.]]
{'AND OR': (array([[0.],
       [0.],
       [0.],
       [1.]], dtype=float32), array([[0.],
       [1.],
       [1.],
       [1.]], dtype=float32)), 'total': array([[0., 0.],
       [0., 1.],
       [1., 0.],
       [2., 2.]], dtype=float32)}


In [5]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

[0.6186204  0.2458607  0.23565328]
[0.44860828 0.7266027  0.4381081 ]
(array([1.9813027, 1.5082669, 1.894467 ], dtype=float32), array([2.9813027, 2.508267 , 2.8944669], dtype=float32))


In [6]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y
print(sess.run(z, feed_dict={x: 3, y: 4.5}))
print(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))

7.5
[3. 7.]


In [7]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()
while True:
  try:
    print(sess.run(next_item))
  except tf.errors.OutOfRangeError:
    break

[0 1]
[2 3]
[4 5]
[6 7]


In [8]:
r = tf.random_normal([10,3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer)
while True:
  try:
    print(sess.run(next_row))
  except tf.errors.OutOfRangeError:
    break

[-0.8896692  -0.43412945 -0.05812366]
[-0.1529216 -1.3058649  0.3406277]
[-0.40093288 -0.34547585  0.8754536 ]
[ 2.4547946  -0.46823183 -1.7718049 ]
[ 0.3701294 -0.0816192 -0.7042009]
[-1.489407  -0.4289316 -1.1567382]
[ 0.02157262 -0.02696867 -0.06085118]
[-0.6720922   0.77427006 -0.23870328]
[-0.18209247 -1.0650271  -0.11241425]
[-1.0220007 -2.0159743 -1.3983421]


In [10]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

init = tf.global_variables_initializer()
sess.run(init)
print(sess.run(y, {x: [[1, 2, 3],[4, 5, 6]]}))

[[2.9877925]
 [7.5702257]]


In [11]:
x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.layers.dense(x, units=1)

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

[[ 4.904642]
 [12.697994]]


In [12]:
features = {
    'sales' : [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']}

department_column = tf.feature_column.categorical_column_with_vocabulary_list(
        'department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)

In [13]:
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))

(None, None)

In [14]:
print(sess.run(inputs))

[[ 1.  0.  5.]
 [ 1.  0. 10.]
 [ 0.  1.  8.]
 [ 0.  1.  9.]]


In [15]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)
linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)

In [16]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y_pred))

[[0.5616628]
 [1.1233256]
 [1.6849884]
 [2.2466512]]


In [19]:
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

print(sess.run(loss))
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

for i in range(100):
  _, loss_value = sess.run((train, loss))
  print(loss_value)

0.1664165
0.1664165
0.1654215
0.1644325
0.16344939
0.16247211
0.16150075
0.16053516
0.15957536
0.15862128
0.15767288
0.15673019
0.15579312
0.15486166
0.15393576
0.15301542
0.15210053
0.15119116
0.15028723
0.14938869
0.1484955
0.14760767
0.14672515
0.1458479
0.1449759
0.14410909
0.1432475
0.14239107
0.14153974
0.14069349
0.13985229
0.13901615
0.13818501
0.1373588
0.13653757
0.1357212
0.13490976
0.13410313
0.1333014
0.13250436
0.13171215
0.13092467
0.13014191
0.1293638
0.12859036
0.1278215
0.1270573
0.12629765
0.12554255
0.124791935
0.124045834
0.12330416
0.12256697
0.12183416
0.1211057
0.12038163
0.119661935
0.118946485
0.118235305
0.11752841
0.116825715
0.116127215
0.1154329
0.11474277
0.11405674
0.11337481
0.112696946
0.11202314
0.1113534
0.110687606
0.11002582
0.109368
0.10871412
0.10806415
0.10741807
0.1067758
0.10613741
0.105502814
0.10487205
0.10424505
0.1036218
0.10300224
0.10238638
0.10177423
0.10116576
0.10056089
0.09995965
0.099362016
0.09876795
0.09817746
0.09759043
0.0970069

In [18]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
for i in range(100):
  _, loss_value = sess.run((train, loss))
  print(loss_value)

print(sess.run(y_pred))

10.01302
7.0388055
4.9745173
3.5416107
2.54681
1.856004
1.3761375
1.0426402
0.810709
0.64925516
0.5367073
0.45809716
0.403039
0.36432612
0.33695775
0.31746438
0.30343807
0.29320836
0.28561604
0.2798566
0.27537194
0.2717747
0.26879618
0.26624978
0.26400623
0.26197556
0.2600955
0.2583227
0.25662714
0.25498796
0.2533907
0.25182524
0.25028458
0.24876392
0.24725966
0.24576966
0.24429211
0.24282593
0.24137019
0.23992439
0.23848805
0.23706079
0.23564254
0.23423307
0.2328322
0.23143984
0.2300559
0.22868025
0.22731291
0.22595376
0.22460277
0.22325987
0.22192502
0.22059815
0.21927923
0.2179682
0.21666497
0.21536958
0.21408188
0.21280196
0.2115296
0.2102649
0.20900774
0.20775814
0.20651597
0.20528127
0.20405392
0.20283395
0.20162125
0.20041578
0.19921751
0.19802642
0.19684243
0.19566557
0.19449571
0.19333285
0.19217695
0.19102794
0.18988582
0.1887505
0.18762204
0.18650025
0.1853852
0.18427683
0.18317504
0.18207987
0.18099122
0.17990914
0.17883348
0.17776425
0.17670141
0.175645
0.17459482
0.173550